# Enable module reloading
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

import datetime
import os
import sys
sys.path.append('..')

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import time;

from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, concatenate
from tensorflow.keras.utils import to_categorical
from src.data.load_data import *

In [19]:
df = load_dataset(filename='../data/Jester-Dataset-ratings.csv')
user_ids, joke_ids, ratings = get_data(df, batch_size=20000)
# batch_size = 20000
        
# matrix = df[0:batch_size].pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)
# user_ids = matrix.index.tolist()
# joke_ids = matrix.columns.tolist()
# ratings = matrix.values.flatten().tolist()
# ratings = np.array(cur_df.pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)).tolist()

In [20]:
user_onehot = to_categorical(user_ids)
joke_onehot = to_categorical(joke_ids)

In [21]:
n_users = len(user_onehot[0])
n_jokes = len(joke_onehot[0])
# n_users = len(np.unique(user_ids))
# n_jokes = len(np.unique(joke_ids))

In [22]:
num_classes = n_users if n_users > n_jokes else n_jokes
user_onehot = to_categorical(user_ids, num_classes=num_classes)
joke_onehot = to_categorical(joke_ids, num_classes=num_classes)

In [23]:
num_classes

511

In [24]:
vector_concat = np.concatenate([user_onehot, joke_onehot], axis=1)

In [25]:
np.array(vector_concat)

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [26]:
np.array(list(zip(user_ids, joke_ids)))

array([[  1,   5],
       [  1,   7],
       [  1,   8],
       ...,
       [510,  68],
       [510,  24],
       [510,  25]])

In [35]:
from src.models.model_GMF import JokeRecommender

model = JokeRecommender(n_users, n_jokes, len(vector_concat))

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01), 
    loss='mean_squared_error', 
#     learning_rate=0.01,
    metrics=['accuracy']
)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=os.path.join("../logs", str(time.time())),
    histogram_freq=1)

model.fit(
    x=np.array(vector_concat),
    y=np.array(ratings),
    batch_size=100,
    epochs=20,
    callbacks=[tensorboard_callback],
    validation_split=0.2
)

Train on 16000 samples, validate on 4000 samples
Epoch 1/20
16000/16000 [==============================] - 2s 135us/sample - loss: 33.1643 - accuracy: 0.0018 - val_loss: 33.1490 - val_accuracy: 7.5000e-04
Epoch 2/20
16000/16000 [==============================] - 1s 88us/sample - loss: 32.9872 - accuracy: 0.0018 - val_loss: 33.0960 - val_accuracy: 7.5000e-04
Epoch 3/20
16000/16000 [==============================] - 1s 92us/sample - loss: 32.9513 - accuracy: 0.0018 - val_loss: 33.0796 - val_accuracy: 7.5000e-04
Epoch 4/20
16000/16000 [==============================] - 1s 91us/sample - loss: 32.9375 - accuracy: 0.0018 - val_loss: 33.0724 - val_accuracy: 7.5000e-04
Epoch 5/20
16000/16000 [==============================] - 1s 87us/sample - loss: 32.9305 - accuracy: 0.0018 - val_loss: 33.0686 - val_accuracy: 7.5000e-04
Epoch 6/20
16000/16000 [==============================] - 1s 86us/sample - loss: 32.9265 - accuracy: 0.0018 - val_loss: 33.0664 - val_accuracy: 7.5000e-04
Epoch 7/20
16000/160

In [36]:
result = model.predict(np.array(vector_concat))

In [37]:
pd.DataFrame(result).describe()

,0
count,2.000000e+04
mean,9.995058e-01
std,3.264762e-09
min,9.995054e-01
25%,9.995055e-01
50%,9.995055e-01
75%,9.995055e-01
max,9.995055e-01


In [38]:
np.column_stack((result, ratings))[:200]

array([[ 0.99950552,  0.219     ],
       [ 0.99950552, -9.281     ],
       [ 0.99950552, -9.281     ],
       [ 0.99950552, -6.781     ],
       [ 0.99950552,  0.875     ],
       [ 0.99950552, -9.656     ],
       [ 0.99950552, -9.031     ],
       [ 0.99950552, -7.469     ],
       [ 0.99950552, -8.719     ],
       [ 0.99950552, -9.156     ],
       [ 0.99950552, -7.188     ],
       [ 0.99950552, -8.781     ],
       [ 0.99950552, -8.531     ],
       [ 0.99950552, -7.906     ],
       [ 0.99950552, -7.469     ],
       [ 0.99950552,  9.812     ],
       [ 0.99950552,  9.906     ],
       [ 0.99950552,  0.75      ],
       [ 0.99950552, -5.        ],
       [ 0.99950552,  2.938     ],
       [ 0.99950552,  2.        ],
       [ 0.99950552, -0.156     ],
       [ 0.99950552,  2.031     ],
       [ 0.99950552,  5.688     ],
       [ 0.99950552,  9.656     ],
       [ 0.99950552,  8.        ],
       [ 0.99950552,  9.312     ],
       [ 0.99950552,  9.312     ],
       [ 0.99950552,